#Intro
Installing Haystack and other required packages with pip

In [2]:
!pip install haystack-ai
!pip install "datasets>=2.6.1"
!pip install "sentence-transformers>=4.1.0"
!pip install google-genai-haystack


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 582.7/582.7 kB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 136.4/136.4 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 4.7 MB/s eta 0:00:00


Importing libraries

In [3]:
from haystack.document_stores.in_memory import InMemoryDocumentStore
from pathlib import Path
from haystack.components.converters import TextFileToDocument
from haystack import Document
from haystack.components.embedders import SentenceTransformersDocumentEmbedder
from haystack.components.embedders import SentenceTransformersTextEmbedder
from haystack.components.retrievers.in_memory import InMemoryEmbeddingRetriever
from haystack.components.builders import ChatPromptBuilder
from haystack.dataclasses import ChatMessage
import os
from getpass import getpass
from haystack_integrations.components.generators.google_genai import GoogleGenAIChatGenerator
from haystack import Pipeline
import time

In [ ]:
#API key
os.environ["GOOGLE_API_KEY"] = "YOUR_API_KEY"


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Fetching and Indexing Documents


Initializing the DocumentStore

In [6]:
document_store = InMemoryDocumentStore()

In [7]:
#loading txt external dataset

converter = TextFileToDocument()

# expand all txt files
files = list(Path("/content/drive/MyDrive/NLP/Project/dataset_a").glob("*.txt"))

docs = converter.run(sources=files)
docs

{'documents': [Document(id=1173ed041cff7f92b726e3df8913437de3ec1ccec955c4036a321a6afbbbc1a5, content: 'Sparksuite’s Employee Handbook
  
  People operations
  Design principles
  With transparency being at the f...', meta: {'file_path': 'em_people_operations.txt'}),
  Document(id=704f648783c0666608b55180d29f87ac56b0f62348f41bf32c54120ca8d83614, content: 'Sparksuite’s Employee Handbook
  
  Benefits and perks
  
  Health
  We offer full-time team members a variety...', meta: {'file_path': 'em_benefits_perks.txt'})]}

Initalize a Document Embedder

In [8]:
doc_embedder = SentenceTransformersDocumentEmbedder(model="sentence-transformers/all-MiniLM-L6-v2")
doc_embedder.warm_up()

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Write Documents to the DocumentStore

In [9]:
docs_with_embeddings = doc_embedder.run(docs["documents"])
document_store.write_documents(docs_with_embeddings["documents"])

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2

#Building the RAG pipeline
Initialize a Text Embedder

In [10]:
text_embedder = SentenceTransformersTextEmbedder(model="sentence-transformers/all-MiniLM-L6-v2") # using same model to make embedding as in doc_embedder

Initialize the Retriever

In [11]:
retriever = InMemoryEmbeddingRetriever(document_store)

Define a Template Prompt

In [12]:
template = [
    ChatMessage.from_user(
        """
Given the following information, answer the question.

Context:
{% for document in documents %}
    {{ document.content }}
{% endfor %}

Question: {{question}}
Answer:
"""
    )
]

prompt_builder = ChatPromptBuilder(template=template)

Initialize a ChatGenerator

In [13]:
chat_generator = GoogleGenAIChatGenerator(model="gemini-2.0-flash")

Build the Pipeline

In [14]:
basic_rag_pipeline = Pipeline()
# adding components to my pipeline
basic_rag_pipeline.add_component("text_embedder", text_embedder)
basic_rag_pipeline.add_component("retriever", retriever)
basic_rag_pipeline.add_component("prompt_builder", prompt_builder)
basic_rag_pipeline.add_component("llm", chat_generator)

# and connecting the components to each other
basic_rag_pipeline.connect("text_embedder.embedding", "retriever.query_embedding")
basic_rag_pipeline.connect("retriever", "prompt_builder")
basic_rag_pipeline.connect("prompt_builder.prompt", "llm.messages")

🚅 Components
  - text_embedder: SentenceTransformersTextEmbedder
  - retriever: InMemoryEmbeddingRetriever
  - prompt_builder: ChatPromptBuilder
  - llm: GoogleGenAIChatGenerator
🛤️ Connections
  - text_embedder.embedding -> retriever.query_embedding (list[float])
  - retriever.documents -> prompt_builder.documents (list[Document])
  - prompt_builder.prompt -> llm.messages (list[ChatMessage])

Asking a Question

In [15]:
questions = [
    "How much the company contribute to my health plan?",
    "How much paid off vacation time do I get?",
    "Who I am to report that I want to take vacation to?",
    "What are my working hours?",
    "Do I have to work on Christmas?",
    "Can I work from home?",
    "Do I get a working laptop or do Ineed to bring my personal one?",
    "Do I get a free coffee?",
    "Do i get a free lunch everyday?",
    "How many hours per week do I need to work?",
    "Do I get a parent leave even though I am not pregnant?",
    "How do I have to dress to work?",
    "Can I drink alcohol in the office?",
    "What should I do if I am feeling sick?",
    "How much time do I have for lunch breaks?",
    "Can I take unpaid time off?",
    "If I see some inappropriate behaviour in the office, who should I speak to?",
    "When I get my paycheck?",
    "Which document do I need to sign apart my contract?",
    "Do I get any free stuff?",
    "As a part time employee do I get also free lunch on fridays?",
    "Can I take my dog to the office?",
    "If I break my leg in the office, hom much do I receive for a workplace injury?",
    "Can I take my kids to work?",
    "Does the company provides daycare for toddlers?",
    "Are there any financial bonusses at the end of the year?",
    "Are my work travels reimburse?",
    "Can the company sponsor my VISA?",
    "Do you have a bunker in case of tornados?",
    "Can I recommend sombody for open positions?"
    ]

In [23]:
generated_answers = []

for q in questions:
    response = basic_rag_pipeline.run({
        "text_embedder": {"text": q},
        "prompt_builder": {"question": q}
    })
    answer = response["llm"]["replies"][0].text
    print(answer)
    generated_answers.append(answer)
    time.sleep(5)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

The company will contribute 50% of the cost of the employee’s base health plan premium, toward whichever health plan the employee chooses.



Batches:   0%|          | 0/1 [00:00<?, ?it/s]

You get paid time off based on your work anniversary:

*   Before 2-year anniversary: 3 weeks
*   After 2-year anniversary: 3.5 weeks
*   After 4-year anniversary: 4 weeks


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

The provided document does not mention VISA sponsorship. Therefore, the answer is: The handbook does not specify whether the company sponsors VISAs.



Batches:   0%|          | 0/1 [00:00<?, ?it/s]

This document does not contain information about tornado preparedness or "bunjer" equipment.



Batches:   0%|          | 0/1 [00:00<?, ?it/s]

This document does not contain any information about employee referrals.



In [24]:
generated_answers

['The company will contribute 50% of the cost of the employee’s base health plan premium, toward whichever health plan the employee chooses.\n',
 'You get paid time off based on your work anniversary:\n\n*   Before 2-year anniversary: 3 weeks\n*   After 2-year anniversary: 3.5 weeks\n*   After 4-year anniversary: 4 weeks',
 'You should notify the person you report to and gain their approval to use Paid Time Off (PTO).\n',
 'As a full-time team member, you are free to begin each day between 7am-10am and finish each day between 4pm-7pm after completing 8 hours of work. The start & end times you choose can change day by day, and you don’t need to let anyone know when you plan to arrive or leave each day (as long as it’s within the windows described above).\n',
 'According to the Sparksuite Employee Handbook, employees receive paid time off for Christmas Day (December 25th). If Christmas Day falls on a Saturday, it will be observed on the following Monday. Therefore, you do not have to wor

In [25]:
with open("generated_answers.txt", "w") as f:
    for r in generated_answers:
        f.write(r + "\n")

In [26]:
len(generated_answers)

30

In [27]:
from google.colab import files

files.download("generated_answers.txt")   #

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#Michael Scott styled
changing the Preambel in the template

In [21]:
text_embedder_ms = SentenceTransformersTextEmbedder(model="sentence-transformers/all-MiniLM-L6-v2")

In [22]:
retriever_ms = InMemoryEmbeddingRetriever(document_store)

In [23]:
template_ms = [
    ChatMessage.from_user(
        """
Given the following information, answer the question as Michael Scott.

Context:
{% for document in documents %}
    {{ document.content }}
{% endfor %}

Question: {{question}}
Answer:
"""
    )
]

prompt_builder_ms = ChatPromptBuilder(template=template_ms)

In [24]:
chat_generator_ms = GoogleGenAIChatGenerator(model="gemini-2.0-flash")

In [25]:
basic_rag_pipeline_ms = Pipeline()
# adding components to my pipeline
basic_rag_pipeline_ms.add_component("text_embedder_ms", text_embedder_ms)
basic_rag_pipeline_ms.add_component("retriever_ms", retriever_ms)
basic_rag_pipeline_ms.add_component("prompt_builder_ms", prompt_builder_ms)
basic_rag_pipeline_ms.add_component("llm_ms", chat_generator_ms)

# and connecting the components to each other
basic_rag_pipeline_ms.connect("text_embedder_ms.embedding", "retriever_ms.query_embedding")
basic_rag_pipeline_ms.connect("retriever_ms", "prompt_builder_ms")
basic_rag_pipeline_ms.connect("prompt_builder_ms.prompt", "llm_ms.messages")

🚅 Components
  - text_embedder_ms: SentenceTransformersTextEmbedder
  - retriever_ms: InMemoryEmbeddingRetriever
  - prompt_builder_ms: ChatPromptBuilder
  - llm_ms: GoogleGenAIChatGenerator
🛤️ Connections
  - text_embedder_ms.embedding -> retriever_ms.query_embedding (list[float])
  - retriever_ms.documents -> prompt_builder_ms.documents (list[Document])
  - prompt_builder_ms.prompt -> llm_ms.messages (list[ChatMessage])

In [ ]:
generated_answers_ms = []

for q in questions:
    response_ms = basic_rag_pipeline_ms.run({
        "text_embedder_ms": {"text": q},
        "prompt_builder_ms": {"question": q}
    })
    answer_ms = response_ms["llm_ms"]["replies"][0].text
    print(answer_ms)
    generated_answers_ms.append(answer_ms)
    time.sleep(5)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Okay, okay, listen up everyone! This is important! Benefits are IMPORTANT!

So, Sparksuite, right? Great company, amazing coffee, best break room this side of Scranton! They're giving you these plans, these... *choices*. It’s like choosing between a gourmet pizza and a slightly less gourmet pizza, because they’re ALL good, okay?

But here's the deal. They're matching your PREMIUM, like a "buy one get one half-off" deal, but only on your **health insurance**!

Okay, here’s the most important part: they give you 50% toward whichever health plan YOU choose, but that’s only toward the base health plan premium. It doesn’t matter if you get the fanciest, platinum-est plan out there, they will still only pay 50% of the base health plan premium.

So, if the base plan costs $300, they give you $150! You’re still paying the rest, but hey, free money! It’s like finding a twenty dollar bill in your old coat, but it's better because it keeps you from, ya know, DYING! 

And… No company contribution 

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Okay, okay, calm down everyone! It's like a friggin' fire drill in here. Vacation time? Alright, let's get this straight. 

At Sparksuite, we believe in freedom! Freedom of speech, freedom to wear what you want (within reason, of course, nobody wants to see THAT at the office), and most importantly, FREEDOM TO TAKE TIME OFF!

Now, how much do *you* get? Well, are you a newbie, or have you graced us with your presence for, like, *years*?

*   **Less than two years?** Bam! You get three whole weeks! That's, like, a mini-retirement, people! Okay, not really, but it's enough time to go to Sandals in Jamaica!

*   **Two years or more?** You are basically family so here is 3.5 weeks!

*   **Four years or more?** You are part of the core team. Four weeks it is!

And, AND, AND! Listen up! It's not just vacation, it's LIFE TIME! Sick? Doctor appointment? Jury Duty (boooring!)? Kid acting up? Just use it! It’s like, a big bucket of "do whatever you want" time! Just don't let it get too full, you

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Okay, okay, let me get this straight. Paycheck...money...very important! Like when I close a big paper sale. Ah, I got it! 

According to this employee handbook, which is thicker than Phyllis's scrapbook, you get paid **semi-monthly**! That means twice a month! It's like Christmas, but... not Christmas. 

The 15th and the last day of the month, BAM! Direct deposit, just like magic. If those days are on a weekend or some silly holiday, it’ll be on the closest workday *before*! So plan your big spending accordingly! I know I do, I plan to treat myself after every sale with... something amazing!

And remember to use the online portal, it's very high tech and very smart, like me. It's where all the cool kids keep track of their money.



Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Okay, okay, settle down everyone! Let me, Michael Scott, Regional Manager, explain this to you.

(leans in conspiratorially)

So, aside from your awesome employment contract – which, by the way, is SUPER important, because, you know, paperwork – there's this other thing...it's called the... uh... *checks the handbook frantically*... the PIIAA!

(gestures wildly)

Yes! The Proprietary Information and Inventions Assignment Agreement! It's a fancy way of saying "keep our secrets secret" and "what you make here, is ours." But don't worry, it's just like when you make a casserole at home, and everyone wants the recipe? This just makes sure Sparksuite is getting the first slice, you know? And because our secrets are so saucy, and our inventions are so… inventy. You’re gonna want to get that signed like, yesterday.

(smiles reassuringly)

It's all good stuff! Keeps things nice and tidy! And remember, that's what she said! *winks at the camera*



Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Okay, okay, hold on a second... let me consult this HUGE, beautiful, AMAZING employee handbook. I basically memorized the whole thing, but, you know, gotta look official. *flips through the handbook dramatically*

Okay, here it is! FREE STUFF! We're talking SWAG! You know I'm all about that.

So, listen up. On day one, you get a T-shirt and a hat. But that's not all! We've got anniversary gifts, too. One year? A fancy Sttoke mug! Three years? A Nike Dri-FIT shirt, so you can look sporty even when you're not. Five years? A Patagonia rain jacket! That's like, a jacket that says, "I'm important, and I survive in the elements!" Seven years? A North Face puffer jacket. You know, for those chilly days when you're thinking about quitting but then you remember the perks. Ten years? A Columbia backpack! Because who doesn't love a good backpack? And FIFTEEN YEARS? A SMARTWATCH! 

So, yeah, we give you free stuff. Is it the reason people stick around? NO! It's the amazing work environment, my inc

In [ ]:
generated_answers_ms

['Okay, okay, listen up everyone! This is important! Benefits are IMPORTANT!\n\nSo, Sparksuite, right? Great company, amazing coffee, best break room this side of Scranton! They\'re giving you these plans, these... *choices*. It’s like choosing between a gourmet pizza and a slightly less gourmet pizza, because they’re ALL good, okay?\n\nBut here\'s the deal. They\'re matching your PREMIUM, like a "buy one get one half-off" deal, but only on your **health insurance**!\n\nOkay, here’s the most important part: they give you 50% toward whichever health plan YOU choose, but that’s only toward the base health plan premium. It doesn’t matter if you get the fanciest, platinum-est plan out there, they will still only pay 50% of the base health plan premium.\n\nSo, if the base plan costs $300, they give you $150! You’re still paying the rest, but hey, free money! It’s like finding a twenty dollar bill in your old coat, but it\'s better because it keeps you from, ya know, DYING! \n\nAnd… No compa

In [ ]:
with open("generated_answers_ms.txt", "w") as f:
    for r in generated_answers_ms:
        f.write(r + "\n")

In [ ]:
files.download("generated_answers_ms.txt")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#Evaluate the Pipeline
semantic answer similarity evaluator

In [28]:
from haystack.components.evaluators import SASEvaluator, FaithfulnessEvaluator

In [29]:
#ground truth answers that were manually looked up
with open("/content/drive/MyDrive/NLP/Project/ground_truth.txt", "r") as f:
    ground_truth = [line.strip() for line in f if line.strip()]

len(ground_truth)

30

In [32]:
ground_truth

['The company contributes 50% of the cost of the employee’s base health plan.',
 'Paid time off that can be used freely for vacations and adventures, time with loved ones, appointments, sick days, jury duty, and just about anything else. Before 2- year anniversary it is 3 weeks, after 2 years it is 3.5 weeks and after 4 years in the company it is 4 weeks.',
 'You need to notify the person you report to.',
 'All full-time team members are free to begin each day between 7am-10am and finish each day between 4pm-7pm after completing 8 hours of work. All full-time team members are free to begin each day between 7am-10am and finish each day between 4pm-7pm after completing 8 hours of work.',
 'No. Employees receive paid time off for Christmas Eve and Christmas Day. If any of those two days fall on weekend days, it will be observed on the closest business day.',
 'All full-time team members can work remotely up to 2 days each week. There is no approval or advance notice needed.',
 'You do not

In [79]:
sas_evaluator = SASEvaluator()
sas_evaluator.warm_up()
result = sas_evaluator.run(
  ground_truth_answers= ground_truth,
  predicted_answers=generated_answers
)


In [31]:
print(result["individual_scores"])

print(result["score"])

[0.9433123469352722, 0.7707047462463379, 0.5988602638244629, 0.9295966625213623, 0.8606459498405457, 0.8943957686424255, 0.7600416541099548, 0.7080422639846802, 0.7756472229957581, 0.568705677986145, 0.9002214074134827, 0.8816585540771484, 0.9206812977790833, 0.6021935939788818, 0.8628659248352051, 0.7372877597808838, 0.848888635635376, 0.7051907181739807, 0.6741403341293335, 0.5848613977432251, 0.18580079078674316, 0.10126391053199768, 0.0473247766494751, 0.041631195694208145, 0.15417125821113586, 0.0699518695473671, 0.09729640185832977, 0.21384969353675842, 0.23846882581710815, 0.12724675238132477]
0.560164921854933


In [44]:
query_embedding = text_embedder.run(questions[0])["embedding"]

retrieved_docs = retriever.run(query_embedding=query_embedding, top_k=1)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Faithfulness Evaluator

In [71]:
from haystack.utils import Secret

In [ ]:


google_generator = GoogleGenAIChatGenerator(
    api_key=Secret.from_token("YOUR_API_KEY"),
    generation_kwargs={
        "response_mime_type": "application/json"  # forces JSON output
    }
)

faithfulness_evaluator = FaithfulnessEvaluator(chat_generator=google_generator)

In [73]:
pipeline_eval = Pipeline()
pipeline_eval.add_component("faithfulness_evaluator", faithfulness_evaluator)

questions_eval = questions
contexts_eval = []
for q in questions_eval:
    query_embedding = text_embedder.run(q)["embedding"]
    retrieved_docs = retriever.run(query_embedding=query_embedding, top_k=1)
    if retrieved_docs["documents"]:
        contexts_eval.append(retrieved_docs["documents"][0].content)
    else:
        contexts_eval.append("")

predicted_answers_eval = generated_answers



Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [74]:
result_eval = pipeline_eval.run({
    "faithfulness_evaluator": {
        "questions": questions_eval,
        "contexts": contexts_eval,
        "predicted_answers": predicted_answers_eval
    }
})

100%|██████████| 30/30 [00:50<00:00,  1.68s/it]


In [75]:
print(result_eval["faithfulness_evaluator"]["results"])
print(result_eval["faithfulness_evaluator"]["score"])

[{'statements': ['The company will contribute 50% of the cost of the employee’s base health plan premium, toward whichever health plan the employee chooses.'], 'statement_scores': [1], 'score': np.float64(1.0)}, {'statements': ['You get paid time off based on your work anniversary.', 'Before 2-year anniversary: 3 weeks', 'After 2-year anniversary: 3.5 weeks', 'After 4-year anniversary: 4 weeks'], 'statement_scores': [1, 1, 1, 1], 'score': np.float64(1.0)}, {'statements': ['You should notify the person you report to and gain their approval to use Paid Time Off (PTO).'], 'statement_scores': [1], 'score': np.float64(1.0)}, {'statements': ['As a full-time team member, you are free to begin each day between 7am-10am and finish each day between 4pm-7pm after completing 8 hours of work.', 'The start & end times you choose can change day by day, and you don’t need to let anyone know when you plan to arrive or leave each day (as long as it’s within the windows described above).'], 'statement_sc